# Ruter

In [7]:
import random
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave_networkx as dnx
import networkx as nx
import dimod


# Number of geolocations (nodes)
num_nodes = 6

# Function to generate random geolocations
def generate_geolocations(num_nodes):
    geolocations = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(num_nodes)]
    return geolocations

# Function to calculate the Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Generate geolocations
geolocations = generate_geolocations(num_nodes)

# Create a complete graph with nodes representing geolocations
G = nx.complete_graph(num_nodes)

# Add edges with weights as the Euclidean distance between geolocations
for i, p1 in enumerate(geolocations):
    for j, p2 in enumerate(geolocations):
        if i != j:
            G.add_edge(i, j, weight=euclidean_distance(p1, p2))

# Set up the QUBO problem
lagrange = 1.0
qubo = dnx.traveling_salesperson_qubo(G, lagrange)

# Convert the QUBO to a BQM (Binary Quadratic Model)
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)

# Set up the sampler
sampler = EmbeddingComposite(DWaveSampler())

# Solve the TSP problem using D-Wave's quantum annealer
sampleset = sampler.sample(bqm)

# Find the shortest path from the sampleset
path = dnx.traveling_salesperson(G, sampleset)

print("Shortest path:", path)
print("Total distance:", sampleset.first.energy)


ValueError: API token not defined